# Value-at-Risk

## Import Libraries

In [1]:

import numpy as np
import pandas as pd
import requests
import itertools
from math import ceil
from time import sleep
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Risk Free Rate 
RF = 0.05
CONF_LEVEL = 0.95


## Historical Value-at-Risk

In [2]:
def get_alphavantagekey(path):
    with open(path) as f:
        key  = f.read().strip()
    return key
key=get_alphavantagekey('keys/alphavantage.txt')

In [66]:
class Security:
    def __init__(self, name, identifier, key, conf_level=0.95):
        self.name = name
        self.identifier = identifier
        self.conf_level = conf_level
        self.get_historical_price_data(key)
        self.HCVaR = self.calc_HistCVaR(conf_level)
        self.HVaR = self.calc_HistVaR(conf_level)

    def __str__(self):
        txt = f"{self.name}({self.identifier})\n"
        txt += f"Historical Returns:{len(self.returns)} | Return: {self.mu*100:.2f}% | Volatility: {self.sigma*100:.2f}%\n"
        txt += f"Historical: VaR: {self.HVaR*100:.2f}% | CVaR: {self.HCVaR*100:.2f}%\n"
        txt += f"Total Return: {self.total_return*100:.2f}%"
        return txt
    
    def calc_HistVaR(self, conf_level):
        """Historical Value-at-Risk for a confidence interval"""
        return np.percentile(self.returns,1-conf_level)

    def calc_HistCVaR(self, conf_level):
        """
        Historical Conditional Value-at-Risk for a confidence interval
        Find the average of the worst returns (below the conf level)
        """

        # number of returns
        n = len(self.returns)

        # sort the returns
        r = np.sort(self.returns)

        # number of returns lower than the confidence level
        num_below_conf = int((1- conf_level) * n)

        # average of the worst returns (below the conf level)
        CVaR = np.mean(r[0:num_below_conf])
        
        return CVaR
    
    def generate_returns(self, mu, sigma, n):
        # init an array of n returns from the normal dist with mean=mu and stdev = sigma
        self.returns = np.random.default_rng().normal(mu, sigma, n)

    def get_historical_price_data(self,key):
        """
        Returns daily data for a stock (symbol)
        key: api key
        symbols: VMW,AAPL,GOOG,SPY
        """
        def calc_return(returns):
            """ calculate annualized return from daily returns"""
            return np.prod(1 + returns) ** (252/len(returns)) - 1
        def calc_volatility(returns):
            """calculate annualized volatilit from daily returns"""
            return np.std(returns) * np.sqrt(252)
        def calc_total_return(self):
            """calculate the total return for the period"""
            index_first = self.prices.index[0]
            index_last = self.prices.index[-1]
            return self.prices.loc[index_last,'close'] / self.prices.loc[index_first, 'close'] -1

        # request data from alphavantage
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={self.identifier}&apikey={key}"
        r = requests.get(url)
        d = r.json()

        # extract data to a df
        df = pd.DataFrame(d['Time Series (Daily)']).T
        df.columns = ['open','high','low','close','volume']
        df['symbol'] = self.identifier

        # change data types
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()

        # convert datatype to float
        for col in ['open','high','low','close','volume']:
            df[col] = df[col].astype('float')

        # calc daily returns
        df['returns'] = df['close'].pct_change()

        # update the security attributes
        self.prices = df
        self.returns = np.array(df['returns'].dropna())
        self.mu = calc_return(self.returns)
        self.sigma = calc_volatility(self.returns)
        self.total_return = calc_total_return(self)
       
    def plot_prices(self):
        df = self.prices
        hover_temp = "Date:%{x}"+ " Closing Price:%{y:,.0f}"
        fig = go.Figure(
            go.Scatter(
                x = df.index,
                y = df.close,
                line = dict(color = 'rgb(83,128,141)',width=2),
                fill='tozeroy',
                fillcolor = 'rgba(83,128,141,0.5)',
                hovertemplate = hover_temp
            )
        )
        fig.update_layout(
            template='plotly_white',
            title = f"{self.name} ({self.identifier}) | Return: {self.mu*100:.2f}% | Volatility: {self.sigma*100:.2f}%",
            yaxis_title='Closing Price' ,
            width=600, height=500
        )
        fig.show()


In [77]:
g = Security('Google','GOOG',key, 0.95)
print(g)
g.plot_prices()



Google(GOOG)
Historical Returns:99 | Return: 46.70% | Volatility: 26.19%
Historical: VaR: -7.21% | CVaR: -4.37%
Total Return: 16.25%


## Monte Carlo Value-at-Risk

## Variance/Covariance Value-at-Risk

## Conditional Value-at-Risk